In [1]:
from package.MicrobeRX.Predictor import Predictor, Analyzer, Visualizer, EnzymeFinder

from package.MicrobeRX.DataBase import DataFiles


import os, multiprocessing

In [ ]:
DataFiles.HUMAN_EVIDENCES

In [2]:
import pandas as pd

In [7]:
Visualizer

<module 'numpy' from '/home/angel/miniconda3/envs/Chem_3/lib/python3.7/site-packages/numpy/__init__.py'>

In [5]:
pd.read_csv(DataFiles.MICROBES_EVIDENCES,sep='\t',compression='gzip')

,source,name,ec,mnx_id,organisms_count,xrefs,origin,complexes_count
0,23DHMPO,"(R)-2,3-Dihydroxy-3-methylpentanoate:NADP+ oxi...",1.1.1.86,MNXR188296,5776,NaN,GutMicrobe,13252.0
1,26DAPLLAT,"L,L-diaminopimelate aminotransferase",2.6.1.83,MNXR97144,1031,NaN,GutMicrobe,1069.0
2,2AHBUTI,(S)-2-Aceto-2-hydroxybutanoate isomerase,1.1.1.86;5.4.99.3,MNXR191966,5696,NaN,GutMicrobe,13163.0
3,2MBCOATA,2-methylbutanoyl-CoA[acyl-carrier-protein] tra...,2.3.1.180,MNXR134855,7088,NaN,GutMicrobe,11183.0
4,3HAD10M12,10-methyl-3-hydroxy-dodecanoyl-ACP hydro-lyase,4.2.1.59,MNXR134870,7020,NaN,GutMicrobe,406.0
...,...,...,...,...,...,...,...,...
7035,TPLOR,theophylline:oxygen oxidoreductase (N3-demethy...,NaN,MNXR111559,3,NaN,GutMicrobe,3.0
7036,CHITIN_DEGepp,"Chitin degradation, assumed extracellular, per...",NaN,NaN,2,NaN,GutMicrobe,0.0
7037,SERR,Serine racemase,NaN,MNXR104348,6,NaN,GutMicrobe,12.0
7038,ANPRTr,"Anthranilate phosphoribosyltransferase, revers...",NaN,NaN,1,NaN,GutMicrobe,0.0


In [ ]:
from pkg_resources import resource_string as resource_bytes

In [ ]:
HUMAN_EVIDENCES=pd.read_csv('../databases/Human/bigg_Recon3D_evidences.tsv',sep='\t')
MICROBIOME_EVIDENCES=pd.read_csv('../databases/GutMicrobiome/microbes_reactions.tsv',sep='\t')
RULES_DATABASE = pd.read_csv('../databases/ReactionRules/ALL_RULES.tsv.gz',sep='\t',compression='gzip')

In [ ]:
HUMAN_EVIDENCES=HUMAN_EVIDENCES[['source', 'name', 'ec', 'mnx_id', 'organisms_count', 'xrefs', 'origin','complexes_count']]
HUMAN_EVIDENCES=HUMAN_EVIDENCES.groupby('mnx_id')['source','name','ec','origin','xrefs'].agg(lambda x: ';'.join([i for i in set(x) if isinstance(i,str)])).reset_index()
HUMAN_EVIDENCES

In [ ]:
reaction_rules=RULES_DATABASE[RULES_DATABASE.reaction_id.isin(HUMAN_EVIDENCES.mnx_id.unique())]

In [ ]:
reaction_rules=pd.concat([reaction_rules,RULES_DATABASE[RULES_DATABASE.reaction_id.isin(MICROBIOME_EVIDENCES.mnx_id.unique())]])
reaction_rules=pd.concat([reaction_rules,RULES_DATABASE[RULES_DATABASE.reaction_id.isin(MICROBIOME_EVIDENCES.source.unique())]])
reaction_rules.drop_duplicates(ignore_index=True,inplace=True)
reaction_rules

In [ ]:
DATA=pd.concat([HUMAN_EVIDENCES[['source', 'name', 'ec', 'mnx_id', 'organisms_count', 'xrefs', 'origin','complexes_count']],MICROBIOME_EVIDENCES])

In [ ]:
DATA=pd.concat([HUMAN_EVIDENCES[['source', 'name', 'ec', 'mnx_id', 'organisms_count', 'xrefs', 'origin','complexes_count']],MICROBIOME_EVIDENCES])
ALL_EVIDENCES=DATA.groupby('mnx_id')['source','name','ec','origin','xrefs'].agg(lambda x: ';'.join([i for i in set(x) if isinstance(i,str)])).reset_index()
ALL_EVIDENCES

In [ ]:
reaction_rules=RULES_DATABASE[RULES_DATABASE.mnx_id.isin(ALL_EVIDENCES.mnx_id.unique())]
reaction_rules

In [ ]:
reaction_rules=reaction_rules[reaction_rules.num_atoms>=2]

In [ ]:
smi="OC1=Nc2ccc(cc2C(=NC1)c1ccccc1)[N+]([O-])=O"
name='Nitrofurantoine'
query=Chem.MolFromSmiles(smi)
query

# Meabolite Predictor

In [ ]:
Predictor=MetabolitePredictor(rules_table=reaction_rules)
Predictor.run_prediction(query,name=name)

In [ ]:
mets=Predictor.unique_metabolites
#mets=mets.merge(ALL_EVIDENCES,on='mnx_id',how='inner')
mets

In [ ]:
mets.reaction_id.unique()

In [ ]:
mets[mets.reaction_id=='CZP_NR']

# Visualizer

In [ ]:
View=Visualizer(mets,query=query,name=name)
View.plot_confidence_scores()

In [ ]:
F=View.plot_evidences(nodes=['mnx_id','origin','name','ec','metabolite_id'])
F

In [ ]:
ma=View.plot_metabolic_accesibility()

In [ ]:
View.display_metabolites(columns_to_display=['mnx_id','source','ec','name'])

# Chemical Analysis

In [ ]:
Analysis=Analyzer(Predictor.unique_metabolites,col_smiles='main_product_smiles')
Analysis.compute_molecular_descriptors()

Analysis.plot_molecular_descriptors()

In [ ]:
Analysis.compute_isotopic_mass()
Analysis.plot_isotopic_masses()

In [ ]:
Analysis.search_pubchem()
Analysis.data_frame

In [ ]:
View=Visualizer(Analysis.data_frame.drop_duplicates(subset='metabolite_id'),query=query,name=name)
View.display_metabolites(legends_col='metabolite_id',columns_to_display=['mnx_id'])

# Omics Integrator

In [ ]:
table=mets[mets.metabolite_id=='Nitrofurantoine_44']
View=Visualizer(table,query=query,name=name)
View.plot_evidences(nodes=['mnx_id','origin','source','name','ec','metabolite_id'])

In [ ]:
sources=[s for entry in table.source for s in entry.split(';')]

In [ ]:
sources

In [ ]:
summary=DATA[DATA.source.isin(sources)][['origin','mnx_id','source','name','organisms_count','complexes_count']].reset_index(drop=True)
summary

In [ ]:
evidence=pd.read_csv('databases/GutMicrobiome/contigs/NTPTP1.tsv',sep='\t')
evidence

In [ ]:
microbes_data=pd.read_csv('databases/GutMicrobiome/microbes_data.tsv',sep='\t')
genomes=microbes_data[~microbes_data['Annotation version ID'].isna()]

genomes_ids=genomes['Annotation version ID'].to_list()

In [ ]:
missing=set(map(str,genomes_ids)) - set([i.replace('.fasta','') for i in os.listdir('databases/GutMicrobiome/proteomes/batch_1/')])
len(missing)